In [ ]:
# pip install crewai
from crewai import Agent, Task, Crew
from crewai.tools import tool
import os

In [ ]:
# The agent asks the developer the program file. 
# {Program}{Language}
# If it's a C++ code, the agent will add the FidesZKP.h library to the user's program.cpp program and will save it as program_AddedFidesZKPLib.cpp 
# If it's a Python code, the agent will add the Fides Python package to the user's program.py program and will save it as program_AddedFidesZKPLib.py 
# {Processor} 
# Then, the agent asks the developer to choose a target processor including RISC-V and ARM. 
# Then, the agent compiles the code using g++ compiler for the target processor.to generate an assembly file; program_AddedFidesZKP.s
# Then, the agent executes the commitmentGenerator file 
# to create program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json, program_AddedFidesZKP_Commitment.json

# The agent returns program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json files to the developer.
# The agent uploads the program_AddedFidesZKP_Commitment.json on the blockchain using https://rpc.fidesinnova.io, API:, with its own wallet.

# The agent return a link to see the submitted commitment on the explorer, https://explorer.fidesinnova.io
# The Fides Innova shows "CommitmentManagerAIAgent" as the submitter on the blockchain, 

In [ ]:
# Role, Goal, Backstory, LLm instance, Tool (optional)
# CompilerTool = tool()
# compilerAgent = Agent(role="commitmentGenerator")
# compileTask = Task()

# Role, Goal, Backstory, LLm instance, Tool (optional)
@tool("CompilerTool")

def compilerTool(programName):
    """Read the user program file, and recognize the program language. Add new lines to the program to import the FidesZKP library or package. Add the '_AddedFidesZKP' to the program name and save it"""
    if ".py" in programName:
        Language = "Python"
    elif ".cpp" in programName:
        Language = "C++"
    else:
        Language = "Unknown"
            
    with open(programName) as f1:
        programCode = f1.read()

    if Language == "C++":
        programCode = "include<fides.h>\m"+programCode
        newFilename = programName.split('.')[0]+"__AddedFidesZKPLib.cpp"
        with open(newFilename, "W+") as f2:
            f2.write(programCode)
        os.system(f"g++ -std=c++17 -S {newFilename} -o {newFilename.split('.')[0]}.s -lstdc++")
        return "The new filename is: " + newFilename.split('.')[0]+".s"
    else:
        raise NotImplementedError    
 
compilerAgent = Agent(role="compiler", goal="Read the user program file {program}, and recognize the program language. Add new lines to the program to import the FidesZKP library or package. Add the '_AddedFidesZKP' to the program name and save it",
                      backstory="", tools = [compilerTool])
compileTask = Task( agent = compilerAgent, description = "process the user's program file {program} with compilerTool.", expected_output="The assembly output filename resulting from executing of the g++ compiler" )

In [ ]:
# Role, Goal, Backstory, LLm instance, Tool (optional)
# commitmentGeneratorTool = tool()
# commitmentGeneratorAgent = Agent(role="commitmentGenerator")
# commitmentGeneratorTask = Task()

# Role, Goal, Backstory, LLm instance, Tool (optional)
@tool("commitmentGeneratorTool")

def commitmentGeneratorTool(programName):
    """Read the user program__AddedFidesZKP.s file, and execute the ./commitmentGenerator file with the program__AddedFidesZKP.s file to generate three files including program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json, and program_AddedFidesZKP_Commitment.json"""
        # Outputs:
        # 1- program_AddedFidesZKP_ZKPGen.s 
        # 2- program_AddedFidesZKP_Param.json 
        # 3- program_AddedFidesZKP_Commitment.json
    if ".s" in programName:
        Language = "assembly"
    else:
        Language = "Unknown"

    if Language == "assembly":
        os.system(f"./commitmentGenerator {programName}")
        return "The new files are: " + programName.split('.')[0]+"_ZKPGen.s," + programName.split('.')[0]+"_Param.json, and "  + programName.split('.')[0]+"_Commitment.json."
    else:
        raise NotImplementedError    
 
 def blockchainUploader(programName):
    """Upload program_AddedFidesZKP_Commitment.json on fides Innova blockchain"""
        # program_AddedFidesZKP_Commitment.json
    if ".s" in programName:
        Language = "assembly"
    else:
        Language = "Unknown"

    if Language == "assembly":
        os.system(f"./commitmentGenerator {programName}")
        return "The transaction id: " + programName.split('.')[0]+"_ZKPGen.s,"
    else:
        raise NotImplementedError   
    
commitmentGeneratorAgent = Agent(role="commitmentGenerator", goal="Read the user program file {program}, and execute commitmentGenerator file. It generates three output files. The firs file is program_AddedFidesZKP_ZKPGen.s that has some extra opcodes compared to the program_AddedFidesZKP.s file. These lines call the proof_generation function from the FidesZKP library or package. The second file is program_AddedFidesZKP_Param.json that contains some pre-calculated numbers to accelerate the proof generation function in production. The third one is a commitment file that shows the arithmetic version of the program, program_AddedFidesZKP_Commitment.json, and should be uploaded on the Fides blockchain. This file will be used later in verification purposes by the proof verifier crew.",
                      backstory="", tools = [commitmentGeneratorTool])
commitmentGeneratorTask = Task( agent = commitmentGeneratorAgent, description = "execute the user's program file {program} with commitmentGenerator, and upload the generated commitment file on Fides Innova blockchain", expected_output="Two files, program_AddedFidesZKP_ZKPGen.s and program_AddedFidesZKP_Param.json to be delivered to the developer. One file, program_AddedFidesZKP_Commitment.json , to be uploaded on the Fides blockchain." )



In [ ]:
# Role, Goal, Backstory, LLm instance, Tool (optional)
# commitmentSubmitterTool = tool()
# commitmentSubmitterAgent = Agent(role="commitmentSubmitter")
# commitmentSubmitterTask = Task()
@tool("commitmentSubmitterTool")

def commitmentSubmitterTool(programName):
    """Read the user program__AddedFidesZKP.s file, and execute the ./commitmentGenerator file with the program__AddedFidesZKP.s file to generate three files including program_AddedFidesZKP_ZKPGen.s, program_AddedFidesZKP_Param.json, and program_AddedFidesZKP_Commitment.json"""
        # Outputs:
        # 1- program_AddedFidesZKP_ZKPGen.s 
        # 2- program_AddedFidesZKP_Param.json 
        # 3- program_AddedFidesZKP_Commitment.json
    if ".s" in programName:
        Language = "assembly"
    else:
        Language = "Unknown"

    if Language == "assembly":
        os.system(f"./commitmentGenerator {programName}")
        return "The new files are: " + programName.split('.')[0]+"_ZKPGen.s," + programName.split('.')[0]+"_Param.json, and "  + programName.split('.')[0]+"_Commitment.json."
    else:
        raise NotImplementedError    
 
 def blockchainUploader(programName):
    """Upload program_AddedFidesZKP_Commitment.json on fides Innova blockchain"""
        # program_AddedFidesZKP_Commitment.json
    if ".s" in programName:
        Language = "assembly"
    else:
        Language = "Unknown"

    if Language == "assembly":
        os.system(f"./commitmentGenerator {programName}")
        return "The transaction id: " + programName.split('.')[0]+"_ZKPGen.s,"
    else:
        raise NotImplementedError   
    
commitmentSubmitterAgent = Agent(role="commitmentGenerator", goal="Read the user program file {program}, and execute commitmentGenerator file. It generates three output files. The firs file is program_AddedFidesZKP_ZKPGen.s that has some extra opcodes compared to the program_AddedFidesZKP.s file. These lines call the proof_generation function from the FidesZKP library or package. The second file is program_AddedFidesZKP_Param.json that contains some pre-calculated numbers to accelerate the proof generation function in production. The third one is a commitment file that shows the arithmetic version of the program, program_AddedFidesZKP_Commitment.json, and should be uploaded on the Fides blockchain. This file will be used later in verification purposes by the proof verifier crew.",
                      backstory="", tools = [commitmentGeneratorTool])
commitmentSubmitterTask = Task( agent = commitmentGeneratorAgent, description = "execute the user's program file {program} with commitmentGenerator, and upload the generated commitment file on Fides Innova blockchain", expected_output="Two files, program_AddedFidesZKP_ZKPGen.s and program_AddedFidesZKP_Param.json to be delivered to the developer. One file, program_AddedFidesZKP_Commitment.json , to be uploaded on the Fides blockchain." )



In [ ]:
# Agent, Description, Expected Output   

agents = [compilerAgent, commitmentGeneratorAgent, commitmentSubmitterAgent]
# agents = [compilerAgent]

task = [compileTask, commitmentGeneratorTask, commitmentSubmitterTask]
# tasks = [compileTask]

crew1 = Crew( agents = agents, tasks= tasks, verbose = True)
crew1.kickoff({"program":"zksensor.cpp"})